<a href="https://colab.research.google.com/github/darwinagudeloh/FDL_2024-2_UDEA/blob/main/03_Preprocesamiento_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar librerías
import requests
from io import StringIO
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
# Cargar el archivo CSV desde GitHub
file_path = "https://raw.githubusercontent.com/JulianaCarvajal/Proyecto_Deep_Learning/main/combined_wind_data.csv"

# Descargar el archivo desde GitHub
response = requests.get(file_path)
if response.status_code == 200:
    # Leer el contenido del archivo en un DataFrame
    data = pd.read_csv(StringIO(response.text))
data = pd.read_csv(file_path)

In [ ]:
# Convertir la columna 'Date' a tipo datetime
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
# Normalización de las Variables
# Seleccionar columnas numéricas para normalización (excluir 'Date')
columns_to_normalize = [col for col in data.columns if 'WS10M_MAX' in col]
scaler = MinMaxScaler(feature_range=(0, 1))
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])

In [ ]:
# Función para Crear Secuencias con Horizontes
sequence_length = 30  # Ventana de 30 días

def create_sequences_with_horizon(data, seq_length, horizon):
    X, y = [], []
    for i in range(len(data) - seq_length - horizon):
        # Entrada: Todas las columnas excepto la columna objetivo
        X.append(data.iloc[i:i + seq_length, :].values)  # Incluye todas las columnas
        # Salida: Solo la columna objetivo desplazada por el horizonte
        y.append(data.iloc[i + seq_length + horizon]['WS10M_MAX_punto6'])
    return np.array(X), np.array(y)

In [ ]:
# Crear Secuencias para Horizonte de 7 Días
horizon_7_days = 7

X_7, y_7 = create_sequences_with_horizon(data[columns_to_normalize], sequence_length, horizon_7_days)

# División en Entrenamiento, Validación y Prueba
train_size_7 = int(0.7 * len(X_7))
val_size_7 = int(0.15 * len(X_7))

X_train_7, y_train_7 = X_7[:train_size_7], y_7[:train_size_7]
X_val_7, y_val_7 = X_7[train_size_7:train_size_7 + val_size_7], y_7[train_size_7:train_size_7 + val_size_7]
X_test_7, y_test_7 = X_7[train_size_7 + val_size_7:], y_7[train_size_7 + val_size_7:]

In [ ]:
# Crear Secuencias para Horizonte de 15 Días
horizon_15_days = 15

X_15, y_15 = create_sequences_with_horizon(data[columns_to_normalize], sequence_length, horizon_15_days)

# División en Entrenamiento, Validación y Prueba
train_size_15 = int(0.7 * len(X_15))
val_size_15 = int(0.15 * len(X_15))

X_train_15, y_train_15 = X_15[:train_size_15], y_15[:train_size_15]
X_val_15, y_val_15 = X_15[train_size_15:train_size_15 + val_size_15], y_15[train_size_15:train_size_15 + val_size_15]
X_test_15, y_test_15 = X_15[train_size_15 + val_size_15:], y_15[train_size_15 + val_size_15:]

In [ ]:
# # Guardar los Datos Procesados
# # Guardar conjuntos de datos
# # Guardar conjuntos para horizonte de 7 días
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/X_train_7.npy', X_train_7)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/y_train_7.npy', y_train_7)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/X_val_7.npy', X_val_7)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/y_val_7.npy', y_val_7)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/X_test_7.npy', X_test_7)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/y_test_7.npy', y_test_7)

# # Guardar conjuntos para horizonte de 15 días
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/X_train_15.npy', X_train_15)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/y_train_15.npy', y_train_15)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/X_val_15.npy', X_val_15)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/y_val_15.npy', y_val_15)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/X_test_15.npy', X_test_15)
# np.save('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/y_test_15.npy', y_test_15)